# Read DWD CDC Temp Time Series

The main idea behind this activity is to see if there is a correlation between temperature and sales of the stores.

In [59]:
import datetime as dt
import os
import ftplib
import pandas as pd
import geopandas as gpd

## FTP Connection

### Connection Parameters

In [60]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

### FTP Directory Definition and Station Description Filename Pattern

In [61]:
# The topic of interest.
topic_dir = "hourly/air_temperature/historical/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
ftp_dir =  ftp_climate_data_dir + topic_dir

### Local Directories

In [62]:
local_ftp_dir         = "data/original/DWD/"      # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir = local_ftp_dir + topic_dir # Local directory where local station info is located
local_ftp_ts_dir      = local_ftp_dir + topic_dir # Local directory where time series downloaded from ftp are located

local_generated_dir   = "data/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir     = local_generated_dir + topic_dir # Derived station data, i.e. the CSV file
local_ts_merged_dir   = local_generated_dir + topic_dir # Parallel merged time series, wide data frame with one TS per column
local_ts_appended_dir = local_generated_dir + topic_dir # Serially appended time series, long data frame for QGIS TimeManager Plugin

In [63]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

data/original/DWD/
data/original/DWD/hourly/air_temperature/historical/
data/original/DWD/hourly/air_temperature/historical/

data/generated/DWD/
data/generated/DWD/hourly/air_temperature/historical/
data/generated/DWD/hourly/air_temperature/historical/
data/generated/DWD/hourly/air_temperature/historical/


In [64]:

os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

### FTP Connect

In [65]:

ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [66]:
ret = ftp.cwd(".")

In [67]:
#ftp.quit()

### FTP Grab File Function

In [68]:
def grabFile(ftpfullname,localfullname):
    try:
        ret = ftp.cwd(".") # A dummy action to chack the connection and to provoke an exception if necessary.
        localfile = open(localfullname, 'wb')
        ftp.retrbinary('RETR ' + ftpfullname, localfile.write, 1024)
        localfile.close()
    
    except ftplib.error_perm:
        print("FTP ERROR. Operation not permitted. File not found?")

    except ftplib.error_temp:
        print("FTP ERROR. Timeout.")

    except ConnectionAbortedError:
        print("FTP ERROR. Connection aborted.")



### Generate Pandas Dataframe from FTP Directory Listing

In [69]:


def gen_df_from_ftp_dir_listing(ftp, ftpdir):
    lines = []
    flist = []
    try:    
        res = ftp.retrlines("LIST "+ftpdir, lines.append)
    except:
        print("Error: ftp.retrlines() failed. ftp timeout? Reconnect!")
        return
        
    if len(lines) == 0:
        print("Error: ftp dir is empty")
        return
    
    for line in lines:
#        print(line)
        [ftype, fsize, fname] = [line[0:1], int(line[31:42]), line[56:]]
#        itemlist = [line[0:1], int(line[31:42]), line[56:]]
#        flist.append(itemlist)
        
        fext = os.path.splitext(fname)[-1]
        
        if fext == ".zip":
            station_id = int(fname.split("_")[2])
        else:
            station_id = -1 
        
        flist.append([station_id, fname, fext, fsize, ftype])
        
        

    df_ftpdir = pd.DataFrame(flist,columns=["station_id", "name", "ext", "size", "type"])
    return(df_ftpdir)

In [70]:
df_ftpdir = gen_df_from_ftp_dir_listing(ftp, ftp_dir)

In [71]:
df_ftpdir.tail(10)

,station_id,name,ext,size,type
618,13777,stundenwerte_TU_13777_20080601_20201231_hist.zip,.zip,606699,-
619,13904,stundenwerte_TU_13904_20080915_20100426_hist.zip,.zip,77723,-
620,13965,stundenwerte_TU_13965_20081201_20201231_hist.zip,.zip,581588,-
621,14003,stundenwerte_TU_14003_19610103_19920629_hist.zip,.zip,1142478,-
622,15000,stundenwerte_TU_15000_20110401_20201231_hist.zip,.zip,466964,-
623,15207,stundenwerte_TU_15207_20131101_20201231_hist.zip,.zip,346831,-
624,15444,stundenwerte_TU_15444_20140901_20201231_hist.zip,.zip,307651,-
625,15555,stundenwerte_TU_15555_20160501_20201231_hist.zip,.zip,230027,-
626,19171,stundenwerte_TU_19171_20200901_20201231_hist.zip,.zip,20827,-
627,19172,stundenwerte_TU_19172_20200901_20201231_hist.zip,.zip,20596,-


### Dataframe with Temp Zip Files

In [72]:
#df_ftpdir["ext"]==".zip"
df_zips = df_ftpdir[df_ftpdir["ext"]==".zip"]
df_zips.set_index("station_id", inplace = True)
df_zips.tail(10)

,name,ext,size,type
station_id,,,,
13777,stundenwerte_TU_13777_20080601_20201231_hist.zip,.zip,606699,-
13904,stundenwerte_TU_13904_20080915_20100426_hist.zip,.zip,77723,-
13965,stundenwerte_TU_13965_20081201_20201231_hist.zip,.zip,581588,-
14003,stundenwerte_TU_14003_19610103_19920629_hist.zip,.zip,1142478,-
15000,stundenwerte_TU_15000_20110401_20201231_hist.zip,.zip,466964,-
15207,stundenwerte_TU_15207_20131101_20201231_hist.zip,.zip,346831,-
15444,stundenwerte_TU_15444_20140901_20201231_hist.zip,.zip,307651,-
15555,stundenwerte_TU_15555_20160501_20201231_hist.zip,.zip,230027,-
19171,stundenwerte_TU_19171_20200901_20201231_hist.zip,.zip,20827,-


### Download the Station Description File

In [73]:
station_fname = df_ftpdir[df_ftpdir['name'].str.contains(station_desc_pattern)]["name"].values[0]
print(station_fname)

# ALternative
#station_fname2 = df_ftpdir[df_ftpdir["name"].str.match("^.*Beschreibung_Stationen.*txt$")]["name"].values[0]
#print(station_fname2)

TU_Stundenwerte_Beschreibung_Stationen.txt


In [74]:
print("grabFile: ")
print("From: " + ftp_dir + station_fname)
print("To:   " + local_ftp_station_dir + station_fname)
grabFile(ftp_dir + station_fname, local_ftp_station_dir + station_fname)

grabFile: 
From: /climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/TU_Stundenwerte_Beschreibung_Stationen.txt
To:   data/original/DWD/hourly/air_temperature/historical/TU_Stundenwerte_Beschreibung_Stationen.txt


In [75]:
# extract column names. They are in German (de)
# We have to use codecs because of difficulties with character encoding (German Umlaute)
import codecs

def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r","utf8")#"utf8", iso8859_2
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en,encoding='iso8859_2', parse_dates=["date_from","date_to"],index_col = 0)
    
    # write csv
    df.to_csv(csvfile, sep = ";")
    return(df)

In [76]:
basename = os.path.splitext(station_fname)[0]
df_stations = station_desc_txt_to_csv(local_ftp_station_dir + station_fname, local_station_dir + basename + ".csv")


df_stations['date_from']=pd.to_datetime(df_stations['date_from'],infer_datetime_format=True) 
df_stations['date_to']=pd.to_datetime(df_stations['date_to'],infer_datetime_format=True)
df_stations.tail()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
15207,2013-11-01,2022-03-09,317,51.2835,9.3590,Schauenburg-Elgershausen,Hessen
15444,2014-09-01,2022-03-09,593,48.4418,9.9216,Ulm-Mähringen,Baden-Württemberg
15555,2016-05-01,2022-03-09,815,47.8761,10.5849,Kaufbeuren-Oberbeuren,Bayern
19171,2020-09-01,2022-03-09,13,54.0038,9.8553,Hasenkrug-Hardebek,Schleswig-Holstein
19172,2020-09-01,2022-03-09,48,54.0246,9.3880,Wacken,Schleswig-Holstein


### Select Stations Located in NRW from Station Description Dataframe

In [77]:
# Create variable with TRUE if state is Nordrhein-Westfalen
isNRW = df_stations['state'] == "Nordrhein-Westfalen"

# Create variable with TRUE if date_to is latest date (indicates operation up to now)
isOperational = df_stations['date_to'] >dt.datetime.strptime('01-01-2019',"%d-%m-%Y")


# select on both conditions
dfNRW = df_stations[isNRW &isOperational]
#print("Number of stations in NRW: \n", dfNRW.count())

print(dfNRW.shape)
dfNRW.tail(2)

(42, 7)


,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
13713,2007-11-01,2022-03-09,386,51.0899,7.6289,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
15000,2011-04-01,2022-03-09,231,50.7983,6.0244,Aachen-Orsbach,Nordrhein-Westfalen


In [78]:
#print(dfNRW.index)
station_ids_selected = list(dfNRW.index)
set(station_ids_selected).issubset(set(df_zips)) # we have some missing stations

False

In [79]:
df_zips.loc[13670,]

name    stundenwerte_TU_13670_20070601_20201231_hist.zip
ext                                                 .zip
size                                              652567
type                                                   -
Name: 13670, dtype: object

In [80]:
station_ids_selected

[390,
 603,
 617,
 1078,
 1246,
 1300,
 1303,
 1327,
 1572,
 1590,
 1766,
 2110,
 2483,
 2497,
 2629,
 2667,
 2947,
 2968,
 3028,
 3031,
 3098,
 3540,
 3591,
 3623,
 4063,
 4127,
 4371,
 5064,
 5347,
 5480,
 5717,
 6197,
 6264,
 6337,
 7106,
 7330,
 7374,
 13670,
 13696,
 13700,
 13713,
 15000]

### Download TS Data from FTP Server

Problem: Not all stations listed in the station description file are associated with a time series (zip file)! The stations in the description file and the set of stations whoch are TS data provided for (zip files) do not match perfectly.  

In [81]:
# Add the names of the zip files only to a list. 
local_zip_list = []

for station_id in station_ids_selected:
    print(station_id)
    try:
        fname = df_zips["name"][station_id]
        #print(fname)
        grabFile(ftp_dir + fname, local_ftp_ts_dir + fname)
        local_zip_list.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

390
603
617
1078
1246
1300
1303
1327
1572
1590
1766
2110
2483
2497
2629
2667
2947
2968
3028
3031
3098
3540
3591
3623
4063
4127
4371
5064
5347
5480
5717
6197
6264
6337
7106
7330
7374
13670
13696
13700
13713
15000


### Concat the Temp Time Series dfs




In [82]:
# column names
#STATIONS_ID;MESS_DATUM;QN_9;TT_TU;RF_TU;eor

In [83]:
import datetime as dt

In [84]:
def temp_ts_to_df(fname):
    
    df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2")
   
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    #print(df.columns)
    df['mess_datum']=pd.to_datetime(df['mess_datum'], format='%Y%m%d%H', utc=True, errors='ignore')
    #begin_date=dt.datetime.strptime('01-08-2019',"%d-%m-%Y")
    begin_date=dt.datetime(2019, 8, 1, 0, 0, 0, tzinfo=dt.timezone.utc)
    df=df[df['mess_datum']>begin_date]

    return(df)

In [85]:
from zipfile import ZipFile

In [86]:

def ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = temp_ts_to_df(myfile)
                df = pd.concat([df, dftmp])

    
    return(df)

In [87]:
df_merged_ts = ts_merge()

Zip archive: data/original/DWD/hourly/air_temperature/historical/stundenwerte_TU_00390_20040701_20201231_hist.zip
Extract product file: produkt_tu_stunde_20040701_20201231_00390.txt



/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: data/original/DWD/hourly/air_temperature/historical/stundenwerte_TU_00603_20010403_20201231_hist.zip
Extract product file: produkt_tu_stunde_20010403_20201231_00603.txt



/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: data/original/DWD/hourly/air_temperature/historical/stundenwerte_TU_00617_20040601_20201231_hist.zip
Extract product file: produkt_tu_stunde_20040601_20201231_00617.txt



...

/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: data/original/DWD/hourly/air_temperature/historical/stundenwerte_TU_13713_20071101_20201231_hist.zip
Extract product file: produkt_tu_stunde_20071101_20201231_13713.txt



/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


Zip archive: data/original/DWD/hourly/air_temperature/historical/stundenwerte_TU_15000_20110401_20201231_hist.zip
Extract product file: produkt_tu_stunde_20110401_20201231_15000.txt



/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/bs/8_rscwxd25z1bcq4d89vsz380000gn/T/ipykernel_82728/4171902304.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


### Description

Stations_ID;Von_Datum;Bis_Datum;Stationsname;Parameter;Parameterbeschreibung;Einheit;Datenquelle (Strukturversion=SV);Zusatz-Info;Besonderheiten;Literaturhinweis;eor;
3;19500401;19930701;Aachen;RF_TU;relative Feuchte;%;Daten der Temperatur und relativen Feuchte (Stundenwerte) aus analogen Registrierungen (Thermohygrograph);Stundenwerte in MEZ;;;eor;
3;19930701;20010401;Aachen;RF_TU;relative Feuchte;%;Daten der Temperatur und relativen Feuchte (Stundenwerte) von automatischen Stationen der 1. Generation (MIRIAM/AFMS2, ESAU-Daten);Stundenwerte in UTC;;;eor;
3;20010401;20110331;Aachen;RF_TU;relative Feuchte;%;Daten der Temperatur und relativen Feuchte (Stundenwerte) generiert aus SYNOP-Meldungen (nach dem 01.04.2001);Stundenwerte in UTC;;;eor;
3;19500401;19930701;Aachen;TT_TU;Lufttemperatur;∞C;Daten der Temperatur und relativen Feuchte (Stundenwerte) aus analogen Registrierungen (Thermohygrograph);Stundenwerte in MEZ;;;eor;
3;19930701;20010401;Aachen;TT_TU;Lufttemperatur;∞C;Daten der Temperatur und relativen Feuchte (Stundenwerte) von automatischen Stationen der 1. Generation (MIRIAM/AFMS2, ESAU-Daten);Stundenwerte in UTC;;;eor;
3;20010401;20110331;Aachen;TT_TU;Lufttemperatur;∞C;Daten der Temperatur und relativen Feuchte (Stundenwerte) generiert aus SYNOP-Meldungen (nach dem 01.04.2001);Stundenwerte in UTC;;;eor;
Legende: FT  = Folgetag; GZ = Gesetzliche Zeit
generiert: 30.03.2021 --  Deutscher Wetterdienst  --


In [88]:
#df_merged_ts['mess_datum']=pd.to_datetime(df_merged_ts['mess_datum'], format='%Y%m%d%H', utc=True, errors='ignore')
df_merged_ts.head()

,stations_id,mess_datum,qn_9,tt_tu,rf_tu,eor
132181,390,2019-08-01 01:00:00+00:00,3,13.0,79.0,eor
132182,390,2019-08-01 02:00:00+00:00,3,12.5,82.0,eor
132183,390,2019-08-01 03:00:00+00:00,3,12.6,83.0,eor
132184,390,2019-08-01 04:00:00+00:00,3,12.7,83.0,eor
132185,390,2019-08-01 05:00:00+00:00,3,12.8,82.0,eor


In [89]:
df_merged_ts.shape

(510645, 6)

In [90]:
df_stations.head(2)

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
3,1950-04-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
44,2007-04-01,2022-03-09,44,52.9336,8.2370,Großenkneten,Niedersachsen


In [91]:
df_all=df_merged_ts.set_index('stations_id').join(df_stations, how='left')

In [92]:
df_merged_ts.to_csv(local_ts_merged_dir + "ts_merged.csv",sep=";")

In [93]:
df_stations.to_csv(local_ts_merged_dir + "df_stations.csv",sep=";")

In [94]:
df_all.head()

,mess_datum,qn_9,tt_tu,rf_tu,eor,date_from,date_to,altitude,latitude,longitude,name,state
390,2019-08-01 01:00:00+00:00,3,13.0,79.0,eor,2004-07-01,2022-03-09,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
390,2019-08-01 02:00:00+00:00,3,12.5,82.0,eor,2004-07-01,2022-03-09,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
390,2019-08-01 03:00:00+00:00,3,12.6,83.0,eor,2004-07-01,2022-03-09,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
390,2019-08-01 04:00:00+00:00,3,12.7,83.0,eor,2004-07-01,2022-03-09,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
390,2019-08-01 05:00:00+00:00,3,12.8,82.0,eor,2004-07-01,2022-03-09,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen


In [95]:
df_all.to_csv(local_ts_merged_dir + "df_all.csv",sep=";")

In [96]:
df_all.shape

(510645, 12)

In [97]:
 df_all['mess_datum'].max()

Timestamp('2020-12-31 23:00:00+0000', tz='UTC')